# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1-Start by creating a new Notebook for this assignment.

2-Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

3-To create the dataframe:
* List The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
* List Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* List More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* List If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* List Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* List In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.



In [1]:
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import lxml
from pandas import DataFrame
import numpy as np

**Obtain the list of postal codes of Canada** 

In [2]:
ca_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
ca_list = requests.get(ca_url)
ca_list

<Response [200]>

**pulling data out of HTML and XML files with Beautiful Soup**

In [3]:
page=soup(ca_list.text,"html.parser")
#page

**Select the table from the HTML**

In [4]:
ca_table=page.table  #will select the <table class="wikitable sortable"> inside html code


In [5]:
#count the number of rows inside the table (ca_table), we need to look the HTML code <tr>
tr=ca_table.find_all('tr')
rows=len(tr)
rows



181

In [6]:
#we have 181 rows with the next format:
tr[0:5]

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>,
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>,
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>,
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>,
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>]

In [7]:
#Let's obtain the headers that are in the first line of code
headers=tr[0].text.split()
headers




['Postal', 'Code', 'Borough', 'Neighborhood']

In [8]:
#We need to extract each row of the table to extract the information into our dataframe.
#let's check the type of information stored for example in line 10
tr[10].text

'\nM1B\n\nScarborough\n\nMalvern, Rouge\n'

In [9]:
#we can split the line using text.split and indicated the line break code \n
tr[10].text.split('\n')

['', 'M1B', '', 'Scarborough', '', 'Malvern, Rouge', '']

In [10]:
#we have a list, and we can extract each information of it:ç
postalcode=tr[10].text.split('\n')[1]
borough=tr[10].text.split('\n')[3]
neighborhood=tr[10].text.split('\n')[5]

print("the postal code is: " + postalcode + " the borough is: " + borough + " and the nb is: " + neighborhood)

the postal code is: M1B the borough is: Scarborough and the nb is: Malvern, Rouge


In [11]:
#now we need to create a while loop to obtain in each row the values of postal code, borough and neghtborhood and store it in a dataframe using pandas
obtain_values =[]
n= 1
while n < rows:
    Postcode=tr[n].text.split('\n')[1]
    Borough=tr[n].text.split('\n')[3]
    Neighborhood=tr[n].text.split('\n')[5]
    obtain_values.append((Postcode, Borough,Neighborhood))
    n=n+1

ca_df=pd.DataFrame(obtain_values, columns=['PostalCode', 'Borough', 'Neighbourhood'])
#note that we also merge the repited postalcodes, like M5A that we can check with: ca_df.head(5)
ca_df.head(5) 


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [12]:
ca_df.shape

(180, 3)

In [13]:
#let's check if is included the complete table checking the last 5 rows
ca_df.tail()

,PostalCode,Borough,Neighbourhood
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,


In [14]:
#drop all cells in Borough and neighbourhood colu& mn with not assigned statement
ca_df=ca_df[(ca_df.Borough != 'Not assigned') | (ca_df.Neighbourhood == 'Not assigned')]
ca_df.reset_index(drop=True, inplace=True) #reset the index after drop
ca_df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
ca_df.shape

(103, 3)